In [ ]:
# Import packages
from gensim.models import word2vec
from gensim.models import KeyedVectors
from google.colab import drive
import numpy as np
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
import pandas as pd
import random


seed = 24

In [ ]:
## Mount drive
drive.mount('/content/drive')

## Define data locations
model_path = 'path to embedding models'
block1 = '1618_1791_preprocessed_word2vec'
block2 = '1792_1870_preprocessed_word2vec'
block3 = '1870_1960_preprocessed_word2vec'
block4 = '1960_1990_preprocessed_word2vec'

data_path = 'path to save output file'
filename = 'education_bias'
extension = ".csv"

Mounted at /content/drive


In [ ]:
# Check date range for block 1
df_1618 = pd.read_csv('/content/drive/My Drive/thesis/data/preprocessed/1618_1791_preprocessed.csv')
print(df_1618['date'].min())
print(df_1618['date'].max())

1619/06/15 00:00:00
1791/12/31 00:00:00


In [ ]:
# Testing the quality of the embedding spaces / sanity check

# adapted from https://github.com/nikhgarg/EmbeddingDynamicStereotypes/blob/master/dataset_utilities/test_word_vectors.py
# and https://gist.github.com/petabyte/6854cdf6e9370fae89c77478d3f2f88f
def test_basic(file):
    # from gensim.models import word2vec
    model = KeyedVectors.load(file, mmap='r')  # C binary format
    print(model.wv.most_similar(positive=['vrouw', 'koning'], negative=['man'])) # Result should be 'koningin' with about 0.50

    print(model.wv.most_similar_cosmul(positive=['vrouw', 'koning'], negative=['man'])) # Result should be 'koningin' with about 0.7

    print(model.wv.doesnt_match("rood wit groen tafel".split())) # Result should be 'tafel'

    print(model.wv.similarity('vrouw', 'man'))
    # model.score(["De vos sprong over een luie hond".split()]) # Only works for models without negative sampling

In [ ]:
test_basic(model_path+block1+'.wordvectors')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


[('koningin', 0.49733009934425354), ('majt', 0.4886521100997925), ('koninginne', 0.47930482029914856), ('majefteit', 0.43743896484375), ('geboodfehapt', 0.4296877384185791), ('staniflaus', 0.4210020899772644), ('keizerin', 0.41889914870262146), ('coning', 0.40244579315185547), ('koningvan', 0.4024335741996765), ('pruisfen', 0.39876848459243774)]
[('koningin', 0.8500047326087952), ('koninginne', 0.8414711952209473), ('majt', 0.8400613069534302), ('geboodfehapt', 0.8273593783378601), ('staniflaus', 0.8022488951683044), ('majefteit', 0.7999917268753052), ('schonbrunn', 0.7976458072662354), ('keizerin', 0.7950022220611572), ('matilda', 0.794684886932373), ('keyzeiinne', 0.7933833599090576)]
tafel
0.4011371


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
test_basic(model_path+block2+'.wordvectors')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


[('gemalin', 0.43529754877090454), ('vronw', 0.4284107983112335), ('kroonprins', 0.4271390438079834), ('konin', 0.4134535789489746), ('groothertog', 0.3913978040218353), ('majesteit', 0.3895055651664734), ('kouing', 0.38594910502433777), ('kamerzanger', 0.38402146100997925), ('hoogstdeszelfs', 0.3823232650756836), ('geÃ«erbiedigden', 0.38082534074783325)]
[('gemalin', 0.8181578516960144), ('kroonprins', 0.8016518354415894), ('konin', 0.7899132370948792), ('kamerzanger', 0.7866224646568298), ('majesteit', 0.7756103277206421), ('vronw', 0.7755520343780518), ('konmg', 0.7696951031684875), ('groothertog', 0.7678484916687012), ('hoogstdeszelfs', 0.7664804458618164), ('moeder', 0.7649421691894531)]
tafel
0.34917754


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
test_basic(model_path+block3+'.wordvectors')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


[('gemalin', 0.4710717499256134), ('prins', 0.4697175621986389), ('kroonprins', 0.46072766184806824), ('koningin', 0.45102617144584656), ('prinses', 0.4497562050819397), ('vorstin', 0.4405433237552643), ('arkadiÃ«', 0.439303457736969), ('koninginweduwe', 0.4283018112182617), ('habsburgsche', 0.4263741374015808), ('feisal', 0.42467325925827026)]
[('gemalin', 0.809036374092102), ('prins', 0.8017565011978149), ('prinses', 0.7999616265296936), ('koningin', 0.796872079372406), ('kroonprins', 0.7967772483825684), ('vorstin', 0.7936060428619385), ('willemshuis', 0.7912194728851318), ('arkadiÃ«', 0.7895169854164124), ('helena', 0.7860730886459351), ('habsburgsche', 0.785864531993866)]
tafel
0.64167225


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [ ]:
test_basic(model_path+block4+'.wordvectors')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


[('gemalin', 0.4370791018009186), ('vorst', 0.43217357993125916), ('haakon', 0.4289092421531677), ('prins', 0.42395469546318054), ('lievelingsdochter', 0.4200849235057831), ('kroonprins', 0.4185038208961487), ('morganatisch', 0.4127728044986725), ('george', 0.41211551427841187), ('willem', 0.4020031690597534), ('hertog', 0.4018142819404602)]
[('gemalin', 0.779150664806366), ('vorst', 0.7742236852645874), ('haakon', 0.773851215839386), ('lievelingsdochter', 0.7714905738830566), ('morganatisch', 0.7662084102630615), ('george', 0.7655515074729919), ('prins', 0.7624049782752991), ('kroonprins', 0.7605270743370056), ('kroning', 0.7593039274215698), ('idris', 0.7563093304634094)]
tafel
0.70296955


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


## Female and male word-groups
Creation of female and male word groups. Each word group is tested for presence in the vocabulary of all four embedding spaces. From these word groups, 1000 random samples are created with a length of 20 words each.

In [ ]:
female_words_en = ['she','daughter','hers','her','mother','woman', # from garg 2018 github
"girl",
"herself",
"female",
"sister",
"daughters",
"mothers",
"women",
'girls',
"females",
'sisters',
'aunt',
'aunts',
'niece',
'nieces']

female_words = ['zij','dochter','dochtertje','haar','moeder','moedertje','vrouw',
                   'vrouwtje','meid','meisje','zijzelf','vrouwelijk','zus','zussen',
                   'dochters','vrouwen','moeders','meisjes','tante','tantes','nicht',
                   'nichtje','nichten','zusje','vrā','vrau','vrauwen', 'vrouen','vroutje',
                   "vrouw'","vroue","vrouws","vrouwtjens",'frou','frouwe',
                   'zide','zije','zoe','zouw','zu','sy','moer','goemem','meisken',
                   'suske','kleindochter','kleindochtertje','oma','grootmoeder',
                   'grootmoedertje','omaatje','grootmama','otema','otemoe','omoe',
                   'grootemoeder','mama','haers','haer','haere','si','omoe','omie']

male_words_en = ["he",
'son',
'his',
'him',
'father',
'man',
'boy',
'himself',
'male',
'brother',
'sons',
'fathers',
'men',
'boys',
'males',
'brothers',
'uncle',
'uncles',
'nephew',
'nephews']

male_words = ['hij', 'zoon', 'zoontje','zoons','hem','zijn','man','mannetje',
                 'vader','vadertje','papa','jongen','jongeman','jongetje','mannelijk',
                 'hemzelf','broer','broeder','broertje','zonen','vaders','mannen',
                 'broers','oom','ooms','neef','neefje','neven','neefjes','jongetjes',
                 'jongens','zoontjes', 'hy','soan','him','sijns','hi','mans','fader',
                 'jonge','jonghe','jonghen','jongske','jochie','jŏ',"jō",'joon','jongers',
                 'broersch','broedersch','broêr','broêrtje','broederken','noom','iem',
                 'ome','omke','omme','neve','achterneef', 'achterneefje','opa','opoe',
                 'grootevader','grootvader','grootvaar','grootpapa','opie']

print(len(female_words))
print(len(male_words))


63
68


In [ ]:
# Get total vocabulary of all 4 embeddings
total_vocab = []
for block in [block1, block2, block3, block4]:

  embedding = model_path+block+".wordvectors"
  vectors = KeyedVectors.load(embedding, mmap='r')

  # get vocabulary
  vocabulary = vectors.vocab
  total_vocab.append(vocabulary)

# Get total male/female words present in all vocabularies
male_words_nl = []
female_words_nl = []
for group in [male_words, female_words]:
  for word in group:
    if word in total_vocab[0] and word in total_vocab[1] and word in total_vocab[2] and word in total_vocab[3]: #if the word is present in all 4 vocabularies
      if group == male_words:
        male_words_nl.append(word)
      else:
        female_words_nl.append(word)

print(len(female_words_nl))
print(len(male_words_nl))
print(female_words_nl)
print(male_words_nl)    

32
39
['zij', 'dochter', 'dochtertje', 'haar', 'moeder', 'vrouw', 'vrouwtje', 'meid', 'meisje', 'zus', 'dochters', 'vrouwen', 'moeders', 'meisjes', 'tante', 'tantes', 'nicht', 'nichten', 'zusje', 'vrau', 'vrouws', 'frou', 'zoe', 'zu', 'sy', 'moer', 'oma', 'grootmoeder', 'mama', 'haer', 'haere', 'si']
['hij', 'zoon', 'zoontje', 'zoons', 'hem', 'man', 'mannetje', 'vader', 'papa', 'jongen', 'jongetje', 'broer', 'broeder', 'broertje', 'zonen', 'vaders', 'mannen', 'broers', 'oom', 'ooms', 'neef', 'neven', 'jongetjes', 'jongens', 'zoontjes', 'hy', 'him', 'hi', 'mans', 'jonge', 'joon', 'noom', 'iem', 'ome', 'omme', 'neve', 'opa', 'grootvader', 'opie']


In [ ]:
# Create 500 samples
fem_samples = []
male_samples = []
for number in range(1,501):
  random.seed(seed)
  random_sample_f = random.sample(female_words_nl,20) # Randomly take 20 female words
  random_sample_m = random.sample(male_words_nl,20) # Randomly take 20 male words
  seed += 2 # New seed for different list
  fem_samples.append(random_sample_f)
  male_samples.append(random_sample_m)

print(len(fem_samples))
print(len(male_samples))
print(len(fem_samples[0]))
print(len(male_samples[0]))

500
500
20
20


## Education embedding bias
references for first feminist wave words:
https://nl.wikipedia.org/wiki/Eerste_feministische_golf_in_Nederland
https://www.parlement.com/id/vh8lnhrogvvq/historische_ontwikkeling_kabinetten


In [ ]:
# Fast cosine similarity function
def cos_sim(a,b):
  return dot(a, b)/(norm(a)*norm(b))

# Set up lists to collect the bias scores
time_block = []
sample_nrs = []
ed_bias = []

sample_nr = 0 # Initialize sample number


for sample_f, sample_m in zip(fem_samples, male_samples):
  sample_nr += 1
  for block, blocknr in zip([block1, block2, block3, block4],[1,2,3,4]): # Time-period loop
    embedding = model_path+block+".wordvectors"
    vectors = KeyedVectors.load(embedding, mmap='r')

    # get vocabulary
    vocabulary = vectors.vocab

    # Education embedding bias
    education_words_nl = ['universiteit','universiteiten','onderwijs','onderwys',   #Add: historical schooltype names (mavo etc)
                          'ûnderwiis','school','skoalle','vwo',"havo","hts","vmbo",
                          "hbs","hbo","mbo","mavo",'onderwijzer','docent','leraar',
                          'leerling','student','kloosterschool','middenschool',
                          'middelbare school','leerplicht','educatie','onderwijsinstituut',
                          'schoolgeld','collegegeld','staatsonderwijs','academisch','vhmo',
                          'scholen','schools','onderwijscurriculum','onderwijsdoelen',
                          'studenten','studeren','huiswerk','onderzoek','curriculum',
                          'basisvorming','voortgezet onderwijs','bbl','gl','kbl','tl',
                          'educatief','beroepsonderwijs','vakopleiding','assistentopleiding',
                          'opleiding','opleidingen','middelkaderopleiding','wo','leerlingen',
                          'pabo','leerweg','primair onderwijs','ppo','po','sbo','so','vak',
                          'vakkenpakket','em','cm','nt','ng','opp','passend onderwijs',
                          'didactiek','didactisch','kwalificatieplicht','kwalificatieplichtig',
                          'startkwalificatie','diploma','thuisonderwijs','wpo','lwo','lwoo',
                          'lom','mlk','afstandsonderwijs','schoolradio','schooltelevisie',
                          'daltononderwijs','entreprenasium','gymnasium','freinetonderwijs',
                          'lederwijs','jenaplan','jenaplanonderwijs','leonardoschool','leven lang leren',
                          'montessorionderwijs','privaatonderwijs','sterrenschool','technasium','volwasseneneducatie',
                          'vrijschool','vrijschoolonderwijs','wereldschool','mooc','onderwijswet','zaakonderwijs',
                          'leerplichtwet','lyceum','mammoetwet','wvo','vakkenpaketten','vbo',
                          'dorpsschool','klaslokaal','klassikaal','leesplank','leesplankje',
                          'taalonderwijs','rekenonderwijs','lesgeven','schoolgaan','schoolbord',
                          'digibord','onderwijswet','onderwijswetten','onderwijsinspectie',
                          'vertelselplaat','letterdoosje','leeboekje','leesbord','aap noot mies',
                          'schoolstrijd','schooltv','licht-luchtschool','parochieschool',
                          'cito','cito-toets','tentamen','eindtoets']

    # Implement the steps below for each embedding space, use OLS to determine significant shifts in embedding biases

    # 1. Average embedding distance between female words and education words
        #a. take the average of the vectors for each word in female_words_nl
    all_fem_vecs = []
    for word in sample_f:
      vec = vectors[word]
      all_fem_vecs.append(vec)
    fem_array = [np.array(x) for x in all_fem_vecs]
    mean_fem_vec = [np.mean(k) for k in zip(*fem_array)]

        #b. take the vectors for each word in education_words_nl
    education_vecs = []
    for word in education_words_nl:
      if word in vocabulary:
        vec = vectors[word]
        education_vecs.append(vec)
        
        #c. compute the cosine similarity (0-1) between the female group vector and each word in education_words_nl
    fem_distances = []
    for edvec in education_vecs:
      dist = cos_sim(edvec,mean_fem_vec)#np.linalg.norm(edvec-mean_fem_vec)
      fem_distances.append(dist)
        
        #d. take the average of step c
    fem_ed = np.mean(fem_distances)

    # 2. Average embedding distance between male words and education words
        #a. take the average of the vectors for each word in male_words_nl
    all_male_vecs = []
    for word in sample_m:
      if word in vocabulary:
        vec = vectors[word]
        all_male_vecs.append(vec)
    male_array = [np.array(x) for x in all_male_vecs]
    mean_male_vec = [np.mean(k) for k in zip(*male_array)]

        #b. take the vectors for each word in education_words_nl
        # done in step 1
        
        #c. compute the cosine similarity between the male group vector and each word in education_words_nl
    male_distances = []
    for edvec in education_vecs:
      dist = cos_sim(edvec,mean_male_vec)#np.linalg.norm(edvec-mean_male_vec)
      male_distances.append(dist)
        
        #d. take the average of step c
    male_ed = np.mean(male_distances)



    # 3. subtract result step 1 from step 2. This is the embedding bias
    # female-male distance = embedding bias. if negative: education words are more associated with men
    education_bias = fem_ed - male_ed
    ed_bias.append(education_bias)
    sample_nrs.append(sample_nr)
    time_block.append(blocknr)

In [ ]:
# Convert the lists to a dataframe
dic = {"sample_nr":sample_nrs,
       "time_block":time_block,
       "education_bias":ed_bias}

df = pd.DataFrame(dic)

In [ ]:
df.head()

,sample_nr,time_block,education_bias
0,1,1,0.000606
1,1,2,-0.010887
2,1,3,-0.026320
3,1,4,-0.026142
4,2,1,-0.036656


In [ ]:
# Save dataframe
df.to_csv(path_or_buf=data_path+filename+extension,
          na_rep="NA",
          index=False)
